## Importing dependencies

In [74]:
import os
from dotenv import load_dotenv
load_dotenv()

True

### Readin Gemini API key

In [75]:
gemini_api_key = os.getenv('GEMINI_API_KEY')

In [76]:
if gemini_api_key:
    print("api key FOUND")
else:
    print("api key NOT FOUND")

api key FOUND


In [77]:
import google.generativeai as genai
from IPython.display import Markdown, display
import llama_index
from llama_index.llms.gemini import Gemini
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings, StorageContext, load_index_from_storage
from llama_index.embeddings.gemini import GeminiEmbedding 

## Pipeline

### Ingestion 

In [78]:
genai.configure(api_key=gemini_api_key)

In [79]:
for model in genai.list_models():
    print(model)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

In [80]:
for model in genai.list_models():
    print(f'Name : {model.name} |--| Generation method : {model.supported_generation_methods}')

Name : models/chat-bison-001 |--| Generation method : ['generateMessage', 'countMessageTokens']
Name : models/text-bison-001 |--| Generation method : ['generateText', 'countTextTokens', 'createTunedTextModel']
Name : models/embedding-gecko-001 |--| Generation method : ['embedText', 'countTextTokens']
Name : models/gemini-1.0-pro-latest |--| Generation method : ['generateContent', 'countTokens']
Name : models/gemini-1.0-pro |--| Generation method : ['generateContent', 'countTokens']
Name : models/gemini-pro |--| Generation method : ['generateContent', 'countTokens']
Name : models/gemini-1.0-pro-001 |--| Generation method : ['generateContent', 'countTokens', 'createTunedModel']
Name : models/gemini-1.0-pro-vision-latest |--| Generation method : ['generateContent', 'countTokens']
Name : models/gemini-pro-vision |--| Generation method : ['generateContent', 'countTokens']
Name : models/gemini-1.5-pro-latest |--| Generation method : ['generateContent', 'countTokens']
Name : models/gemini-1.5

In [81]:
documents = SimpleDirectoryReader("../Data")

In [82]:
doc = documents.load_data()

In [83]:
print(doc[0].text)

Indian Institute of Technology dhArwAD (IITdh) is one of the third Generation (3-G) IITs, which were established by the Ministry of Education (MoE), Government of India (GoI). It was started in August 2016 under the mentorship of IIT Bombay. Our academic journey began in the Academic Year (AY) 2016-17, with the establishment of four-year Bachelor of Technology (B. Tech.) programs - Mechanical Engineering (ME), Electrical Engineering (EE) and Computer Science & Engineering (CSE).

View our Vision & Mission

Currently our Institute offers four other B. Tech. programs - Engineering Physics (EP), Mathematics & Computing (M&C), Civil & Infrastructure Engineering (C&IE), Chemical & Biochemical Engineering (C&BE) as well as a five-year Bachelor of Science-Master of Science (BS-MS) program, a two-year Master of Technology (M. Tech.) Program and a Master of Science (MS) Program offered by the three oldest Departments of Mechanical, Materials & Aerospace Engineering (MMAE), Electrical, Electroni

In [84]:
model = Gemini(model='models/gemini-pro', api_key=gemini_api_key)
print(model)

callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x773a606ce030> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x773a87f193a0> completion_to_prompt=<function default_completion_to_prompt at 0x773a87f8f4c0> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None model='models/gemini-pro' temperature=0.1 max_tokens=2048 generate_kwargs={}


### Creating Embeddings

In [85]:
gemini_embed_model = GeminiEmbedding(
    model_name='models/embedding-001',
    api_key=gemini_api_key
)

### Creating ServiceContext

In [86]:
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.node_parser = SentenceSplitter(chunk_size=800, chunk_overlap=20)


### Creating VectorStore

In [87]:
index = VectorStoreIndex(doc, embed_model=gemini_embed_model)

In [88]:
index.storage_context.persist()

In [89]:
query_engine = index.as_query_engine()

### Quering from VectorStore

In [92]:
response = query_engine.query("Full form of IIT Dh")
print(response.response)

Indian Institute of Technology dhArwAD


In [93]:
response = query_engine.query("what are the B.tech programs IIT Dharwad is offering")
print(response.response)

- Mechanical Engineering (ME)
- Electrical Engineering (EE)
- Computer Science & Engineering (CSE)
- Engineering Physics (EP)
- Mathematics & Computing (M&C)
- Civil & Infrastructure Engineering (C&IE)
- Chemical & Biochemical Engineering (C&BE)


In [95]:
response = query_engine.query("give me some information of student there")
print(response.response)

IIT dhArwAD accommodates around 1,075 students, with around 20% being women. The diverse student body hails from 20 states/ Union territories (UTs) out of the 36 states/ UTs across the country. Among them, around 20% are pursuing Masters and Doctoral degrees.


In [97]:
response = query_engine.query("from which historical place IIT Dharwad is inspired from")
print(response.response)

Hubballi dhArwAD


In [98]:
response = query_engine.query("tell me some deep insights of IIT Dharwad")
print(response.response)

IIT Dharwad has a diverse student body from 20 states/ Union territories (UTs) out of the 36 states/ UTs across the country. Around 20% of the students are pursuing Masters and Doctoral degrees. The institute has carefully planned and built a world-class infrastructure in a sprawling area of 470 acres [encompassing 67 acres of natural forest], which will be net zero in terms of water, energy, and waste. IIT Dharwad has strategically brought on board a faculty with a global exposure, emphasizing on high-quality research and interdisciplinary programs. The institute collaborates in thrust areas like net zero (i.e., sustainable) physical infrastructure, Cyber-Physical Systems, Green Mobility, Precision Agri-Tech, 3-D Printing for Additive/Subtractive Manufacturing. Noteworthy initiatives include a Space Data Science Lab (SDSL) and two Global Centres of Excellence [GCoE] in Affordable and Clean Energy (ACE) and Indian Knowledge Systems (IKS).
